In [ ]:
## converting streamflows into km3 
import numpy as np

def convert_m3s_to_km3month(flow_m3s):
    # Conversion factors
    seconds_per_minute = 60
    minutes_per_hour = 60
    hours_per_day = 24
    days_per_month = 30.44
    
    # Calculate seconds per month
    seconds_per_month = seconds_per_minute * minutes_per_hour * hours_per_day * days_per_month
    
    # Convert m3/s to km3/month
    flow_km3month = flow_m3s * seconds_per_month / (10**9)
    
    return flow_km3month

#Load the data from text files
Qgs = np.loadtxt('meltwater_hist.txt')  # Assuming this data is in m³/s
rainbase_data = np.loadtxt('rainbase.txt')  # Assuming this data is in m³/s
total=np.loadtxt('total.txt')

# Convert data
Qgs_converted = convert_m3s_to_km3month(Qgs)
rainbase_converted = convert_m3s_to_km3month(rainbase_data)
total_converted = convert_m3s_to_km3month(total)

# Optionally, save the converted data back to text files
np.savetxt('meltwater_hist_km3.txt', Qgs_converted)
np.savetxt('rainbase_km3.txt', rainbase_converted)
np.savetxt('total_km3.txt', total_converted)

print("Conversion complete and files saved.")


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.stats import qmc

def assure_path_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Created directory: {path}")

def generate_demand_flows(n_scenarios, lhs_samples):
    demand_param_ranges = {
        "Cotton": [6.2844, 106.5503],
        "Rice": [0.000000, 86.1922],
        "Wheat": [0.7141, 97.3107],
        "Sugarcane": [1.2042, 162.1384],
        "Miscellaneous": [3.3697, 47.7030],
    }
    crop_percentages = {
        'Cotton': [0.00, 0.00, 0.00, 0.00, 0.10, 0.15, 0.20, 0.20, 0.15, 0.15, 0.05, 0.00],
        'Rice': [0.00, 0.00, 0.00, 0.00, 0.00, 0.25, 0.25, 0.20, 0.20, 0.10, 0.00, 0.00],
        'Wheat': [0.20, 0.25, 0.25, 0.15, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.15],
        'Sugarcane': [0.083] * 12,
        'Miscellaneous': [0.083] * 12
    }
    
    demand_flows = {}
    total_demand = np.zeros((n_scenarios, 12))
    
    for i, crop in enumerate(demand_param_ranges.keys()):
        annual_demand = lhs_samples[:, i] 
        monthly_demands = np.outer(annual_demand, crop_percentages[crop])
        demand_flows[crop] = monthly_demands
        total_demand += monthly_demands
        print(f"{crop} Demand (km^3): Min = {np.min(annual_demand):.3f}, Max = {np.max(annual_demand):.3f}")
        print(f"{crop} Monthly Sum (km^3): Min = {np.min(np.sum(monthly_demands, axis=1)):.3f}, Max = {np.max(np.sum(monthly_demands, axis=1)):.3f}")
    
    demand_flows['Total'] = total_demand
    print(f"Total Annual Demand (km^3): Min = {np.min(np.sum(total_demand, axis=1)):.3f}, Max = {np.max(np.sum(total_demand, axis=1)):.3f}")
    
    return demand_flows

def lhs_params(param_ranges, n_samples):
    n_params = len(param_ranges)
    sampler = qmc.LatinHypercube(d=n_params)
    lhs_samples = sampler.random(n=n_samples)
    for i in range(n_params):
        min_val, max_val = param_ranges[i]
        lhs_samples[:, i] = min_val + (max_val - min_val) * lhs_samples[:, i]
    return lhs_samples

def ComponentRescaling(sites, HistoricalMonthlyQ, LHsamples, rescaled_folder):
    mMonths = 12
    nScenarios = LHsamples.shape[0]
    nYears = len(HistoricalMonthlyQ) // mMonths

    historical_avg = np.mean(HistoricalMonthlyQ.reshape(nYears, mMonths), axis=0)
    
    for j in range(nScenarios):
        amplitude, lambda_coeff = LHsamples[j, :]
        
        NewMonthlyQ = np.zeros(len(HistoricalMonthlyQ))
        for m in range(12):
            k = (m + 3) % 12
            q_new = amplitude * (lambda_coeff * historical_avg[k] + (1 - lambda_coeff) * historical_avg[m])
            NewMonthlyQ[m::mMonths] = q_new
        
        for site in sites:
            file_path = os.path.join(rescaled_folder, f"{site}_Scenario{j+1}.csv")
            np.savetxt(file_path, NewMonthlyQ, fmt='%.3f', delimiter=',')
        
        scaling_factors = NewMonthlyQ[:mMonths] / historical_avg
        scaling_factors_path = os.path.join(rescaled_folder, f"Scenario{j+1}_scaling_factors.txt")
        np.savetxt(scaling_factors_path, scaling_factors, fmt='%.3f')

def CombineRescaledWithRainbase(sites, nScenarios, rescaled_folder, rainbase_data, combined_folder):
    nYears = 30
    mMonths = 12
    total_months = nYears * mMonths
    
    if len(rainbase_data) < total_months:
        rainbase_data = np.tile(rainbase_data, (total_months // len(rainbase_data) + 1))[:total_months]
    
    rainbase_reshaped = rainbase_data.reshape(nYears, mMonths)
    
    for i in range(nScenarios):
        for site in sites:
            rescaled_data = np.loadtxt(os.path.join(rescaled_folder, f"{site}_Scenario{i+1}.csv"), delimiter=',')
            
            if rescaled_data.shape != (nYears, mMonths):
                rescaled_data = rescaled_data.reshape(nYears, mMonths)
            
            combined_data = rescaled_data + rainbase_reshaped
            file_path = os.path.join(combined_folder, f"total_{site}_Scenario{i+1}.csv")
            np.savetxt(file_path, combined_data, fmt='%.3f', delimiter=',')

def plot_supply_flows(combined_folder, plots_folder):
    assure_path_exists(plots_folder)
    
    for filename in os.listdir(combined_folder):
        if filename.startswith("total_") and filename.endswith(".csv"):
            file_path = os.path.join(combined_folder, filename)
            data = np.loadtxt(file_path, delimiter=',')
            
            plt.figure(figsize=(12, 6))
            plt.plot(data.flatten())
            plt.title(f"Supply Flow: {filename}")
            plt.xlabel("Month")
            plt.ylabel("Flow ($km^3$)")
            plt.savefig(os.path.join(plots_folder, f"{filename[:-4]}_plot.png"))
            plt.close()

def calculate_scenario_averages(combined_folder, averages_folder):
    assure_path_exists(averages_folder)
    
    for filename in os.listdir(combined_folder):
        if filename.startswith("total_") and filename.endswith(".csv"):
            file_path = os.path.join(combined_folder, filename)
            data = np.loadtxt(file_path, delimiter=',')
            
            # Calculate the average for each month across all years
            average_data = np.mean(data, axis=0)
            
            # Save the average data
            average_file_path = os.path.join(averages_folder, f"avg_{filename}")
            np.savetxt(average_file_path, average_data, fmt='%.3f', delimiter=',')

def plot_supply_demand_flows(averages_folder, average_plots_folder, demand_folder):
    assure_path_exists(average_plots_folder)
    
    crop_colors = {
        'Cotton': 'yellow',
        'Rice': 'purple',
        'Wheat': 'blue',
        'Sugarcane': 'red',
        'Miscellaneous': 'cyan',
        'Total': 'black'
    }

    # Load demand data
    demand_data = {}
    for crop in crop_colors.keys():
        file_path = os.path.join(demand_folder, f"{crop}_demand.csv")
        demand_data[crop] = np.loadtxt(file_path, delimiter=',')
        print(f"Loaded {crop} demand data. Shape: {demand_data[crop].shape}")

    supply_files = sorted([f for f in os.listdir(averages_folder) if f.startswith("avg_") and f.endswith(".csv")])
    
    for scenario_index, supply_file in enumerate(supply_files):
        supply_path = os.path.join(averages_folder, supply_file)
        supply_data = np.loadtxt(supply_path, delimiter=',')
        print(f"Supply data shape for scenario {scenario_index + 1}: {supply_data.shape}")
        
        plt.figure(figsize=(12, 6))
        
        # Plot supply data
        plt.plot(supply_data, label='Supply', color='green', linewidth=2)
        
        # Calculate supply statistics
        supply_avg = np.mean(supply_data)
        supply_min = np.min(supply_data)
        supply_max = np.max(supply_data)
        
        # Plot demand data for each crop and total
        for crop, color in crop_colors.items():
            if demand_data[crop].ndim == 1:
                crop_demand = demand_data[crop]
            else:
                crop_demand = demand_data[crop][scenario_index] if scenario_index < demand_data[crop].shape[0] else demand_data[crop][0]
            
            print(f"Plotting {crop} demand. Shape: {crop_demand.shape}, Min: {np.min(crop_demand)}, Max: {np.max(crop_demand)}")
            
            if crop == 'Total':
                plt.plot(crop_demand, label=crop, color=color, linestyle='--', linewidth=2)
            else:
                plt.plot(crop_demand, label=crop, color=color)
        
        plt.title(f"Average Supply and Demand Flows: Scenario {scenario_index + 1}")
        plt.xlabel("Month")
        plt.ylabel("Average Flow (km^3)")
        plt.xticks(range(12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        
        # Add supply statistics to the plot
        plt.text(0.02, 0.98, f"Supply Avg: {supply_avg:.2f} km^3\nSupply Min: {supply_min:.2f} km^3\nSupply Max: {supply_max:.2f} km^3", 
                 transform=plt.gca().transAxes, verticalalignment='top', 
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        plt.savefig(os.path.join(average_plots_folder, f"scenario_{scenario_index + 1}_with_demand_plot.png"))
        plt.close()

    print("Supply and demand flows plotted with supply statistics.")

# Main execution
sites = ['All']
n_scenarios = 1000

demand_param_ranges = {
    "Cotton": [6.2844, 106.5503],
    "Rice": [0.000000, 86.1922],
    "Wheat": [0.7141, 97.3107],
    "Sugarcane": [1.2042, 162.1384],
    "Miscellaneous": [3.3697, 47.7030],
}

all_param_ranges = list(demand_param_ranges.values()) + [[0.47, 1.78], [0, 1]]  # Amplitude and lambda

all_lhs_samples = lhs_params(all_param_ranges, n_scenarios)
np.savetxt("LHsamples_all_params.txt", all_lhs_samples, fmt='%.6f')

supply_lhs_samples = all_lhs_samples[:, -2:]

demand_flows = generate_demand_flows(n_scenarios, all_lhs_samples[:, :5])

Qgs = np.loadtxt('meltwater_hist_km3.txt')
rainbase_data = np.loadtxt('rainbase_km3.txt')

supply_flows_folder = './debugged'
rescaled_folder = os.path.join(supply_flows_folder, '1. rescaled_meltwater')
combined_folder = os.path.join(supply_flows_folder, '2. combined')
#plots_folder = os.path.join(supply_flows_folder, '3. supply_flow_plots') # no need bogus 
averages_folder = os.path.join(supply_flows_folder, '5. supply_averages')
average_plots_folder = os.path.join(supply_flows_folder, '6. average_supply_flow_plots') # takes very long -_- 100 plots but mandatory check

for folder in [supply_flows_folder, rescaled_folder, combined_folder, plots_folder, averages_folder, average_plots_folder]:
    assure_path_exists(folder)

ComponentRescaling(sites, Qgs, supply_lhs_samples, rescaled_folder)
CombineRescaledWithRainbase(sites, n_scenarios, rescaled_folder, rainbase_data, combined_folder)
plot_supply_flows(combined_folder, plots_folder)

# Save demand flows
demand_folder = os.path.join(supply_flows_folder, '4. demand_flows')
assure_path_exists(demand_folder)
for crop, flows in demand_flows.items():
    np.savetxt(os.path.join(demand_folder, f"{crop}_demand.csv"), flows, delimiter=',')


print("Demand flows shapes:")
for crop, flows in demand_flows.items():
    print(f"{crop}: {flows.shape}")
    
# Calculate and save scenario averages
calculate_scenario_averages(combined_folder, averages_folder)

# Plot supply and demand flows for each scenario 
plot_supply_demand_flows(averages_folder, average_plots_folder, demand_folder)

print("Step 1 completed: Component Rescaling, Adding Rainbase, and Generating Demand Flows")
print("Step 2 completed: Calculated scenario averages and plotted average flows")


Cotton Demand (km^3): Min = 6.328, Max = 106.500
Cotton Monthly Sum (km^3): Min = 6.328, Max = 106.500
Rice Demand (km^3): Min = 0.047, Max = 86.116
Rice Monthly Sum (km^3): Min = 0.047, Max = 86.116
Wheat Demand (km^3): Min = 0.786, Max = 97.274
Wheat Monthly Sum (km^3): Min = 0.786, Max = 97.274
Sugarcane Demand (km^3): Min = 1.334, Max = 162.106
Sugarcane Monthly Sum (km^3): Min = 1.329, Max = 161.458
Miscellaneous Demand (km^3): Min = 3.370, Max = 47.687
Miscellaneous Monthly Sum (km^3): Min = 3.357, Max = 47.497
Total Annual Demand (km^3): Min = 84.585, Max = 443.518
Created directory: ./debugged
Created directory: ./debugged\1. rescaled_meltwater
Created directory: ./debugged\2. combined
Created directory: ./debugged\3. supply_flow_plots
Created directory: ./debugged\5. supply_averages
Created directory: ./debugged\6. average_supply_flow_plots
Created directory: ./debugged\4. demand_flows
Demand flows shapes:
Cotton: (1000, 12)
Rice: (1000, 12)
Wheat: (1000, 12)
Sugarcane: (1000,

In [2]:
#step # 2
def create_time_series_plots_with_crops(supply_data, demand_flows, results_folder):
    assure_path_exists(results_folder)
    
    n_months = 12
    months = range(1, n_months + 1)
    
    plt.figure(figsize=(12, 8))
    
    # Plot supply range and average
    supply_max = np.max(supply_data, axis=0)
    supply_min = np.min(supply_data, axis=0)
    supply_avg = np.mean(supply_data, axis=0)
    
    plt.fill_between(months, supply_min, supply_max, alpha=0.2, color='gray', label='Supply Range')
    plt.plot(months, supply_avg, label='Average Supply', color='gray', linewidth=2)
    
    # Plot demand for each crop
    crops = ['Cotton', 'Rice', 'Wheat', 'Sugarcane', 'Miscellaneous']
    colors = ['yellow', 'purple', 'blue', 'red', 'cyan']
    
    for crop, color in zip(crops, colors):
        crop_demand_min = np.min(demand_flows[crop], axis=0)
        crop_demand_max = np.max(demand_flows[crop], axis=0)
        crop_demand_avg = np.mean(demand_flows[crop], axis=0)
        
        plt.fill_between(months, crop_demand_min, crop_demand_max, alpha=0.2, color=color)
        plt.plot(months, crop_demand_avg, label=f'{crop} Demand', color=color, linewidth=2)

    # Calculate and plot total demand
    total_demand_data = np.sum([demand_flows[crop] for crop in crops], axis=0)
    total_demand_min = np.min(total_demand_data, axis=0)
    total_demand_max = np.max(total_demand_data, axis=0)
    total_demand_avg = np.mean(total_demand_data, axis=0)
    
    plt.fill_between(months, total_demand_min, total_demand_max, alpha=0.2, color='black')
    plt.plot(months, total_demand_avg, label='Total Demand', color='black', linewidth=2, linestyle='--')

    plt.xlabel('Month')
    plt.ylabel('Water Volume (km³)')
    plt.title('Supply and Demand Time Series')
    plt.xticks(months, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    plt.legend(bbox_to_anchor=(0.5, -0.15), loc='upper center', ncol=4)
    plt.tight_layout()
    plt.savefig(os.path.join(results_folder, 'supply_demand_time_series_with_crops.png'), bbox_inches='tight')
    plt.close()

    print(f"Time series plot has been saved in {results_folder}")

    # Print summary statistics
    print("\nSummary Statistics (km³):")
    print(f"Supply - Min: {supply_min.min():.4f}, Max: {supply_max.max():.4f}, Avg: {supply_avg.mean():.4f}")
    print(f"Total Demand - Min: {total_demand_min.min():.4f}, Max: {total_demand_max.max():.4f}, Avg: {total_demand_avg.mean():.4f}")
    for crop in crops:
        crop_demand_min = np.min(demand_flows[crop], axis=0)
        crop_demand_max = np.max(demand_flows[crop], axis=0)
        crop_demand_avg = np.mean(demand_flows[crop], axis=0)
        print(f"{crop} Demand - Min: {crop_demand_min.min():.4f}, Max: {crop_demand_max.max():.4f}, Avg: {crop_demand_avg.mean():.4f}")

# Prepare data for time series plots
supply_data = np.array([np.loadtxt(os.path.join(averages_folder, f)) for f in os.listdir(averages_folder) if f.startswith("avg_")])

time_series_folder = os.path.join(supply_flows_folder, '7. time_series_plots')
assure_path_exists(time_series_folder)

create_time_series_plots_with_crops(supply_data, demand_flows, time_series_folder)
print("Step 2 completed: Plotting ranges for Supply Flows vs Demand Flows and Time Series Plot")


Created directory: ./debugged\7. time_series_plots
Time series plot has been saved in ./debugged\7. time_series_plots

Summary Statistics (km³):
Supply - Min: 2.0990, Max: 46.5620, Avg: 11.8630
Total Demand - Min: 1.6123, Max: 56.6270, Avg: 21.2752
Cotton Demand - Min: 0.0000, Max: 21.3000, Avg: 4.7014
Rice Demand - Min: 0.0000, Max: 21.5291, Avg: 3.5913
Wheat Demand - Min: 0.0000, Max: 24.3186, Avg: 4.0844
Sugarcane Demand - Min: 0.1107, Max: 13.4548, Avg: 6.7787
Miscellaneous Demand - Min: 0.2797, Max: 3.9581, Avg: 2.1195
Step 2 completed: Plotting ranges for Supply Flows vs Demand Flows and Time Series Plot


In [13]:
def calculate_var_metrics(supply_data, demand_flows, storage_capacity):
    n_scenarios = len(supply_data)
    
    annual_scarcity = np.zeros(n_scenarios)
    surplus_capacity = np.zeros(n_scenarios)
    deficit_severity = np.zeros(n_scenarios)
    variability = np.zeros(n_scenarios)
    best_months = np.zeros(n_scenarios, dtype=int)
    worst_months = np.zeros(n_scenarios, dtype=int)
    storage_adequacy = np.zeros(n_scenarios)
    
    for i in range(n_scenarios):
        print(f"Processing scenario {i+1}/{n_scenarios}")
        print(f"Supply data shape for this scenario: {supply_data[i].shape}")
        
        if supply_data[i].ndim == 1:
            monthly_supply = supply_data[i]
        else:
            monthly_supply = np.mean(supply_data[i], axis=0)
        
        print(f"Monthly supply shape: {monthly_supply.shape}")
        
        monthly_demand = np.sum([demand_flows[crop][i] for crop in demand_flows if crop != 'Total' and crop != 'All Scenarios Total'], axis=0)
        print(f"Monthly demand shape: {monthly_demand.shape}")
        
        # Calculate annual scarcity
        annual_supply = np.sum(monthly_supply)
        annual_demand = np.sum(monthly_demand)
        annual_scarcity[i] = annual_demand / annual_supply
        
        if len(monthly_supply) < 3:
            print(f"Warning: monthly_supply for scenario {i} has less than 3 elements. Skipping rolling calculations.")
            continue
        
        extended_supply = np.concatenate([monthly_supply[-2:], monthly_supply, monthly_supply[:2]])
        extended_demand = np.concatenate([monthly_demand[-2:], monthly_demand, monthly_demand[:2]])
        
        # Calculate 3-month moving averages
        rolling_supply = np.convolve(extended_supply, np.ones(3), 'valid') /3
        rolling_demand = np.convolve(extended_demand, np.ones(3), 'valid') /3
        
        # Calculate rolling balance (average supply minus average demand)
        rolling_balance = rolling_supply - rolling_demand
        
        # Calculate variability
        best_balance = np.max(rolling_balance)
        worst_balance = np.min(rolling_balance)
        variability[i] = best_balance - worst_balance
        
        # Find best and worst months
        best_months[i] = np.argmax(rolling_balance) % 12
        worst_months[i] = np.argmin(rolling_balance) % 12
        
        # Calculate surplus capacity
        surplus_capacity[i] = best_balance
        
        # Calculate deficit severity
        deficit_severity[i] = -worst_balance
        
        # Calculate storage adequacy
        storage_adequacy[i] = storage_capacity / variability[i]
    
    return annual_scarcity, variability, surplus_capacity, deficit_severity, best_months, worst_months, storage_adequacy
def plot_lambda_vs_amplitude_updated_again(data, metrics, output_file):
    descriptive_labels = {
        'Annual_Scarcity': 'Annual Scarcity',
        'Deficit_Severity_km3': 'Deficit Severity ($km^3$)',
        'Surplus_Capacity_km3': 'Surplus Capacity ($km^3$)',
        'Variability_km3': 'Variability ($km^3$)',
        'Storage_Adequacy_Met': 'Storage Adequacy'
    }
    fig, axs = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Lambda vs Amplitude', fontsize=16)
    
    axs = axs.flatten()
    subplot_labels = ['(a)', '(b)', '(c)', '(d)', '(e)']
    
    for i, metric in enumerate(metrics):
        ax = axs[i]
        
        if metric in ['Annual_Scarcity', 'Deficit_Severity_km3', 'Surplus_Capacity_km3', 'Variability_km3']:
            scatter = ax.scatter(data['Input_lambda'], data['Input_Amplitude'], 
                                 c=data[metric], cmap='coolwarm', s=50)
            ax.set_title(descriptive_labels[metric])
            cbar = plt.colorbar(scatter, ax=ax)
            cbar.set_label(descriptive_labels[metric], rotation=270, labelpad=15)
        
        elif metric == 'Storage_Adequacy_Met':
            colors = ['red' if val == 0 else '#1f77b4' for val in data[metric]]
            scatter = ax.scatter(data['Input_lambda'], data['Input_Amplitude'], c=colors, s=50)
            ax.set_title('Storage Adequacy')
            
            # Create custom legend
            from matplotlib.lines import Line2D
            legend_elements = [Line2D([0], [0], marker='o', color='w', label='Not Met (0)',
                                      markerfacecolor='red', markersize=10),
                               Line2D([0], [0], marker='o', color='w', label='Met (1)',
                                      markerfacecolor='#1f77b4', markersize=10)]
            ax.legend(handles=legend_elements, loc='best')
        
        ax.set_xlabel('Lambda')
        ax.set_ylabel('Amplitude')
        ax.text(0.03, 1.07, subplot_labels[i], transform=ax.transAxes, fontsize=12,
                verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', edgecolor='none', facecolor='white'))
    
    # Remove the last (empty) subplot
    fig.delaxes(axs[5])
    
    plt.tight_layout()
    plt.savefig(output_file, bbox_inches='tight', dpi=600)
    plt.close()

    print(f"Lambda vs Amplitude plot saved to {output_file}")


# Calculate VAR metrics
storage_capacity = 32.3
storage_adequacy_threshold = 1.0  # Set your desired threshold here
annual_scarcity, variability, surplus_capacity, deficit_severity, best_months, worst_months, storage_adequacy = calculate_var_metrics(supply_data, demand_flows, storage_capacity)



# Prepare data for lambda vs amplitude plot
data = pd.DataFrame({
    'Input_lambda': supply_lhs_samples[:, 1],
    'Input_Amplitude': supply_lhs_samples[:, 0],
    'Annual_Scarcity': annual_scarcity,
    'Variability_km3': variability,
    'Surplus_Capacity_km3': surplus_capacity,
    'Deficit_Severity_km3': deficit_severity,
    'Storage_Adequacy': storage_adequacy,
    'Storage_Adequacy_Met': storage_adequacy_met
})

data.to_csv(os.path.join(var_metrics_folder,'complete_results.csv'))
# Create lambda vs amplitude plot
var_metrics_folder = os.path.join(supply_flows_folder, '8. VAR_metrics')
assure_path_exists(var_metrics_folder)
output_file = os.path.join(var_metrics_folder, 'lambda_vs_amplitude.png')
metrics_to_plot = ['Annual_Scarcity', 'Variability_km3', 'Surplus_Capacity_km3', 'Deficit_Severity_km3', 'Storage_Adequacy_Met']
plot_lambda_vs_amplitude_updated_again(data, metrics_to_plot, output_file)

Processing scenario 1/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 2/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 3/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 4/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 5/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 6/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 7/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 8/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape

In [35]:
def classify_storage_adequacy(variability, current_storage, potential_storage):
    scores = np.zeros(len(variability), dtype=int)
    scores[(variability <= current_storage)] = 1
    scores[(variability > current_storage) & (variability <= potential_storage)] = 2
    return scores

def calculate_var_metrics(supply_data, demand_flows, storage_capacity):
    n_scenarios = len(supply_data)
    
    annual_scarcity = np.zeros(n_scenarios)
    surplus_capacity = np.zeros(n_scenarios)
    deficit_severity = np.zeros(n_scenarios)
    variability = np.zeros(n_scenarios)
    best_months = np.zeros(n_scenarios, dtype=int)
    worst_months = np.zeros(n_scenarios, dtype=int)
    storage_adequacy = np.zeros(n_scenarios)
    
    for i in range(n_scenarios):
        print(f"Processing scenario {i+1}/{n_scenarios}")
        print(f"Supply data shape for this scenario: {supply_data[i].shape}")
        
        if supply_data[i].ndim == 1:
            monthly_supply = supply_data[i]
        else:
            monthly_supply = np.mean(supply_data[i], axis=0)
        
        print(f"Monthly supply shape: {monthly_supply.shape}")
        
        monthly_demand = np.sum([demand_flows[crop][i] for crop in demand_flows if crop != 'Total' and crop != 'All Scenarios Total'], axis=0)
        print(f"Monthly demand shape: {monthly_demand.shape}")
        
        # Calculate annual scarcity
        annual_supply = np.sum(monthly_supply)
        annual_demand = np.sum(monthly_demand)
        annual_scarcity[i] = annual_demand / annual_supply
        
        if len(monthly_supply) < 3:
            print(f"Warning: monthly_supply for scenario {i} has less than 3 elements. Skipping rolling calculations.")
            continue
        
        extended_supply = np.concatenate([monthly_supply[-2:], monthly_supply, monthly_supply[:2]])
        extended_demand = np.concatenate([monthly_demand[-2:], monthly_demand, monthly_demand[:2]])
        
        # Calculate 3-month moving averages
        rolling_supply = np.convolve(extended_supply, np.ones(3), 'valid') /3
        rolling_demand = np.convolve(extended_demand, np.ones(3), 'valid') /3
        
        # Calculate rolling balance (average supply minus average demand)
        rolling_balance = rolling_supply - rolling_demand
        
        # Calculate variability
        best_balance = np.max(rolling_balance)
        worst_balance = np.min(rolling_balance)
        variability[i] = best_balance - worst_balance
        
        # Find best and worst months
        best_months[i] = np.argmax(rolling_balance) % 12
        worst_months[i] = np.argmin(rolling_balance) % 12
        
        # Calculate surplus capacity
        surplus_capacity[i] = best_balance
        
        # Calculate deficit severity
        deficit_severity[i] = -worst_balance
        
        # Calculate storage adequacy
        storage_adequacy[i] = storage_capacity / variability[i]
    
    return annual_scarcity, variability, surplus_capacity, deficit_severity, best_months, worst_months, storage_adequacy

def plot_lambda_vs_amplitude_updated_again(data, metrics, output_file):
    descriptive_labels = {
        'Annual_Scarcity': 'Annual Scarcity',
        'Deficit_Severity_km3': 'Deficit Severity ($km^3$)',
        'Surplus_Capacity_km3': 'Surplus Capacity ($km^3$)',
        'Variability_km3': 'Variability ($km^3$)',
        'Storage_Adequacy': 'Storage Adequacy'
    }
    fig, axs = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Lambda vs Amplitude', fontsize=16)
    
    axs = axs.flatten()
    subplot_labels = ['(a)', '(b)', '(c)', '(d)', '(e)']
    
    for i, metric in enumerate(metrics):
        ax = axs[i]
        
        if metric in ['Annual_Scarcity', 'Deficit_Severity_km3', 'Surplus_Capacity_km3', 'Variability_km3']:
            scatter = ax.scatter(data['Input_lambda'], data['Input_Amplitude'], 
                                 c=data[metric], cmap='coolwarm', s=50)
            ax.set_title(descriptive_labels[metric])
            cbar = plt.colorbar(scatter, ax=ax)
            cbar.set_label(descriptive_labels[metric], rotation=270, labelpad=15)
        
        elif metric == 'Storage_Adequacy':
            colors = ['red' if val == 0 else ('#1f77b4' if val == 1 else (1, 1, 0, 0.5)) for val in data[metric]]
            #colors = ['red' if val == 0 else ('#1f77b4' if val == 1 else 'yellow') for val in data[metric]]
            scatter = ax.scatter(data['Input_lambda'], data['Input_Amplitude'], c=colors, s=50)
            ax.set_title('Storage Adequacy')
            
            # Create custom legend
            from matplotlib.lines import Line2D
            legend_elements = [Line2D([0], [0], marker='o', color='w', label='Not Met (0)',
                                      markerfacecolor='red', markersize=10),
                               Line2D([0], [0], marker='o', color='w', label='Current (1)',
                                      markerfacecolor='#1f77b4', markersize=10),
                               Line2D([0], [0], marker='o', color='w', label='Potential (2)',
                                      markerfacecolor='yellow', markersize=10)]
            ax.legend(handles=legend_elements, loc='best')
        
        ax.set_xlabel('Lambda')
        ax.set_ylabel('Amplitude')
        ax.text(0.03, 1.07, subplot_labels[i], transform=ax.transAxes, fontsize=12,
                verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', edgecolor='none', facecolor='white'))
    
    # Remove the last (empty) subplot
    fig.delaxes(axs[5])
    
    plt.tight_layout()
    plt.savefig(output_file, bbox_inches='tight', dpi=600)
    plt.close()

    print(f"Lambda vs Amplitude plot saved to {output_file}")


# Calculate VAR metrics
current_storage = 14.3
potential_storage = current_storage + 18.0  # 32.3 BCM in total

annual_scarcity, variability, surplus_capacity, deficit_severity, best_months, worst_months, _ = calculate_var_metrics(supply_data, demand_flows, potential_storage)

# Classify storage adequacy
storage_adequacy = classify_storage_adequacy(variability, current_storage, potential_storage)

# Prepare data for lambda vs amplitude plot
data = pd.DataFrame({
    'Input_lambda': supply_lhs_samples[:, 1],
    'Input_Amplitude': supply_lhs_samples[:, 0],
    'Annual_Scarcity': annual_scarcity,
    'Variability_km3': variability,
    'Surplus_Capacity_km3': surplus_capacity,
    'Deficit_Severity_km3': deficit_severity,
    'Storage_Adequacy': storage_adequacy
})

# Create lambda vs amplitude plot
var_metrics_folder = os.path.join(supply_flows_folder, '8. VAR_metrics')
assure_path_exists(var_metrics_folder)
output_file = os.path.join(var_metrics_folder, 'lambda_vs_amplitude_storage_updated_2_trial.png')
metrics_to_plot = ['Annual_Scarcity', 'Variability_km3', 'Surplus_Capacity_km3', 'Deficit_Severity_km3', 'Storage_Adequacy']
plot_lambda_vs_amplitude_updated_again(data, metrics_to_plot, output_file)

# Save VAR metrics to CSV
data.to_csv(os.path.join(var_metrics_folder, 'VAR_metrics_updated.csv'), index=False)

print("Step 3 completed: Calculated VAR metrics and plotted Lambda vs Amplitude")



Processing scenario 1/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 2/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 3/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 4/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 5/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 6/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 7/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape: (12,)
Processing scenario 8/1000
Supply data shape for this scenario: (12,)
Monthly supply shape: (12,)
Monthly demand shape

C:\Users\HP\anaconda3\envs\osti\lib\site-packages\matplotlib\cbook\__init__.py:1072: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.asanyarray(x)


Lambda vs Amplitude plot saved to ./debugged\8. VAR_metrics\lambda_vs_amplitude_storage_updated_2_trial.png
Step 3 completed: Calculated VAR metrics and plotted Lambda vs Amplitude


In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

def assure_path_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Created directory: {path}")

def plot_correlation_matrix(data, output_folder):
    assure_path_exists(output_folder)
    
    # Select the columns for correlation
    columns_to_correlate = [
        'Annual_Scarcity', 'Variability_km3', 'Surplus_Capacity_km3', 'Deficit_Severity_km3', 'Storage_Adequacy',
        'Cotton_Demand', 'Rice_Demand', 'Wheat_Demand', 'Sugarcane_Demand', 'Miscellaneous_Demand',
        'Input_Amplitude', 'Input_lambda'
    ]
    
    correlation_data = data[columns_to_correlate]
    
    # Compute the correlation matrix
    corr_matrix = correlation_data.corr()
    
    # Create a heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, fmt='.2f')
    plt.title('Correlation Matrix of Parameters and Metrics')
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'Correlation_Matrix.png'), dpi=300)
    plt.close()
    
    print(f"Correlation matrix has been saved to {output_folder}")

# Main execution for step 4
supply_flows_folder = './debugged'  # Adjust this path as needed
var_metrics_folder = os.path.join(supply_flows_folder, '8. VAR_metrics')
correlation_folder = os.path.join(supply_flows_folder, '9. correlation')

# Load VAR metrics
var_metrics_file = os.path.join(var_metrics_folder, 'VAR_metrics.csv')
if not os.path.exists(var_metrics_file):
    print(f"Error: VAR metrics file not found at {var_metrics_file}")
    print("Current working directory:", os.getcwd())
    print("Contents of the VAR metrics folder:")
    print(os.listdir(var_metrics_folder))
    raise FileNotFoundError(f"VAR metrics file not found: {var_metrics_file}")

metrics_df = pd.read_csv(var_metrics_file)

# Load demand data
demand_folder = os.path.join(supply_flows_folder, '4. demand_flows')
crops = ['Cotton', 'Rice', 'Wheat', 'Sugarcane', 'Miscellaneous']
for crop in crops:
    file_path = os.path.join(demand_folder, f"{crop}_demand.csv")
    if not os.path.exists(file_path):
        print(f"Warning: Demand file for {crop} not found at {file_path}")
        continue
    demand_data = np.loadtxt(file_path, delimiter=',')
    metrics_df[f'{crop}_Demand'] = np.mean(demand_data, axis=1)

# Load input parameters
input_params_file = "LHsamples_all_params.txt"
if not os.path.exists(input_params_file):
    print(f"Error: Input parameters file not found at {input_params_file}")
    raise FileNotFoundError(f"Input parameters file not found: {input_params_file}")

input_params = np.loadtxt(input_params_file)
metrics_df['Input_Amplitude'] = input_params[:, -2]
metrics_df['Input_lambda'] = input_params[:, -1]

# Plot correlation matrix
plot_correlation_matrix(metrics_df, correlation_folder)

print("Step 4 completed: Plotting Correlation Matrix")

Created directory: ./debugged\9. correlation
Correlation matrix has been saved to ./debugged\9. correlation
Step 4 completed: Plotting Correlation Matrix


In [26]:
import numpy as np
import matplotlib.pyplot as plt
import os

def assure_path_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Created directory: {path}")

def create_scarcity_variability_plots(annual_scarcity, variability, all_lhs_samples, all_param_ranges, output_folder):
    assure_path_exists(output_folder)

    
    param_names = [
        "Cotton Demand", "Rice Demand", "Wheat Demand", "Sugarcane Demand",
        "Miscellaneous Demand"
    ]
    
    for i, param_name in enumerate(param_names):
        plt.figure(figsize=(10, 6))
        scatter = plt.scatter(annual_scarcity, variability, c=all_lhs_samples[:, i], cmap='coolwarm')
        plt.xlabel("Annual Water Scarcity")
        plt.ylabel("Monthly Variability (km³)")
        plt.title(f"Normalized Water Scarcity vs Monthly Variability\nColor: {param_name}")
        
        cbar = plt.colorbar(scatter)
        cbar.set_label(f'{param_name} Value')
        # Set colorbar limits to the parameter range
        cbar.mappable.set_clim(all_param_ranges[i])
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f"scarcity_vs_variability_{param_name.replace(' ', '_').lower()}.png"), dpi=300, bbox_inches='tight')
        plt.close()
    
    print(f"Scarcity vs Variability plots have been saved to {output_folder}")

# Main execution
supply_flows_folder = './debugged'  # Adjust this path as needed
var_metrics_folder = os.path.join(supply_flows_folder, '8. VAR_metrics')
scarcity_variability_folder = os.path.join(supply_flows_folder, '11. scarcity_variability_plots')

# Load VAR metrics
var_metrics_file = os.path.join(var_metrics_folder, 'VAR_metrics.csv')
metrics_df = pd.read_csv(var_metrics_file)

# Extract annual scarcity and variability from metrics_df
annual_scarcity = metrics_df['Annual_Scarcity'].values
variability = metrics_df['Variability_km3'].values

# Load LHS samples
all_lhs_samples = np.loadtxt("LHsamples_all_params.txt")

# Define parameter ranges
all_param_ranges = [
    [6.2844, 106.5503],  # Cotton
    [0.000000, 86.1922],  # Rice
    [0.7141, 97.3107],  # Wheat
    [1.2042, 162.1384],  # Sugarcane
    [3.3697, 47.7030],  # Miscellaneous
    [0.47, 1.78],  # Amplitude
    [0, 1]  # Lambda
]

# Create scarcity vs variability plots
create_scarcity_variability_plots(annual_scarcity, variability, all_lhs_samples, all_param_ranges, scarcity_variability_folder)

print("Step 6 completed: Created Scarcity vs Variability plots")

Scarcity vs Variability plots have been saved to ./debugged\11. scarcity_variability_plots
Step 6 completed: Created Scarcity vs Variability plots


In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import itertools

def assure_path_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Created directory: {path}")




def plot_supply_demand_combinations(all_lhs_samples, normalized_scarcity, results_folder):
    assure_path_exists(results_folder)  # Ensure the directory exists

    supply_params = ['Amplitude', 'lambda']
    demand_params = ['Cotton', 'Rice', 'Wheat', 'Sugarcane', 'Miscellaneous']
    
    # Create all combinations of supply and demand parameters
    combinations = list(itertools.product(supply_params, demand_params))
    
    # Set up the plot grid
    n_plots = len(combinations)
    n_cols = 5  # You can adjust this for a different layout
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 5*n_rows))
    fig.suptitle('Supply vs Demand Parameters (Color: Normalized Water Scarcity)', fontsize=16)
    
    for i, (supply_param, demand_param) in enumerate(combinations):
        row = i // n_cols
        col = i % n_cols
        ax = axs[row, col] if n_rows > 1 else axs[col]
        
        supply_index = supply_params.index(supply_param) + len(demand_params)
        demand_index = demand_params.index(demand_param)
        
        scatter = ax.scatter(all_lhs_samples[:, supply_index], 
                             all_lhs_samples[:, demand_index],
                             c=normalized_scarcity, 
                             cmap='coolwarm')
        
        ax.set_xlabel(supply_param)
        ax.set_ylabel(demand_param)
        ax.set_title(f'{supply_param} vs {demand_param}')
        
        fig.colorbar(scatter, ax=ax, label='Normalized Water Scarcity')
    
    # Remove any unused subplots
    for i in range(n_plots, n_rows * n_cols):
        row = i // n_cols
        col = i % n_cols
        fig.delaxes(axs[row, col] if n_rows > 1 else axs[col])
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(results_folder, 'supply_demand_combinations.png'))
    plt.close()



def plot_crop_vs_parameter(crop_demands, parameter_values, parameter_name, normalized_metrics, output_folder):
    metrics = ['annual_scarcity', 'variability', 'surplus_capacity', 'deficit_severity', 'storage_adequacy']
    crops = ['Cotton', 'Rice', 'Wheat', 'Sugarcane', 'Miscellaneous']
    
    for crop_index, crop in enumerate(crops):
        for metric in metrics:
            plt.figure(figsize=(10, 6))
            scatter = plt.scatter(crop_demands[:, crop_index], parameter_values, c=normalized_metrics[metric], cmap='coolwarm')
            plt.colorbar(scatter, label=f'Normalized {metric.replace("_", " ").title()}')
            plt.xlabel(f'{crop} Demand')
            plt.ylabel(parameter_name)
            plt.title(f'{crop} Demand vs {parameter_name} (Color: {metric.replace("_", " ").title()})')
            plt.savefig(os.path.join(output_folder, f'{crop.lower()}_vs_{parameter_name.lower()}_{metric}.png'))
            plt.close()


# Main execution
supply_flows_folder = './debugged'  # Adjust this path as needed
var_metrics_folder = os.path.join(supply_flows_folder, '8. VAR_metrics')
scarcity_variability_folder = os.path.join(supply_flows_folder, '11. scarcity_variability_plots')
supply_demand_folder = os.path.join(supply_flows_folder, '12. supply_demand_plots')
crop_parameter_folder = os.path.join(supply_flows_folder, '13. crop_parameter_plots')

# Ensure all directories exist
for folder in [var_metrics_folder, scarcity_variability_folder, supply_demand_folder, crop_parameter_folder]:
    assure_path_exists(folder)

# Load VAR metrics
var_metrics_file = os.path.join(var_metrics_folder, 'VAR_metrics.csv')
metrics_df = pd.read_csv(var_metrics_file)

# Extract metrics from metrics_df
annual_scarcity = metrics_df['Annual_Scarcity'].values
variability = metrics_df['Variability_km3'].values
surplus_capacity = metrics_df['Surplus_Capacity_km3'].values
deficit_severity = metrics_df['Deficit_Severity_km3'].values
storage_adequacy = metrics_df['Storage_Adequacy'].values

# Normalize metrics
normalized_scarcity = (annual_scarcity - np.min(annual_scarcity)) / (np.max(annual_scarcity) - np.min(annual_scarcity))
normalized_metrics = {
    'annual_scarcity': normalized_scarcity,
    'variability': (variability - np.min(variability)) / (np.max(variability) - np.min(variability)),
    'surplus_capacity': (surplus_capacity - np.min(surplus_capacity)) / (np.max(surplus_capacity) - np.min(surplus_capacity)),
    'deficit_severity': (deficit_severity - np.min(deficit_severity)) / (np.max(deficit_severity) - np.min(deficit_severity)),
    'storage_adequacy': (storage_adequacy - np.min(storage_adequacy)) / (np.max(storage_adequacy) - np.min(storage_adequacy))
}

# Load LHS samples
all_lhs_samples = np.loadtxt("LHsamples_all_params.txt")

# Define parameter ranges
all_param_ranges = [
    [6.2844, 106.5503],  # Cotton
    [0.000000, 86.1922],  # Rice
    [0.7141, 97.3107],  # Wheat
    [1.2042, 162.1384],  # Sugarcane
    [3.3697, 47.7030],  # Miscellaneous
    [0.47, 1.78],  # Amplitude
    [0, 1]  # Lambda
]


# Create supply vs demand plots
plot_supply_demand_combinations(all_lhs_samples, normalized_scarcity, supply_demand_folder)

# Create crop vs parameter plots
crop_demands = all_lhs_samples[:, :5]  # First 5 columns are crop demands
for i, param_name in enumerate(['Amplitude', 'Lambda']):
    param_values = all_lhs_samples[:, 5+i]
    plot_crop_vs_parameter(crop_demands, param_values, param_name, normalized_metrics, crop_parameter_folder)

print("Step 6 completed: Created Scarcity vs Variability plots")
print("Step 7 completed: Created Supply vs Demand plots")
print("Step 8 completed: Created Crop vs Parameter plots")

Created directory: ./debugged\12. supply_demand_plots
Created directory: ./debugged\13. crop_parameter_plots
Step 6 completed: Created Scarcity vs Variability plots
Step 7 completed: Created Supply vs Demand plots
Step 8 completed: Created Crop vs Parameter plots


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import itertools

def assure_path_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Created directory: {path}")

def plot_supply_demand_combinations(all_lhs_samples, annual_scarcity, results_folder):
    assure_path_exists(results_folder)

    supply_params = ['Amplitude', 'lambda']
    demand_params = ['Cotton', 'Rice', 'Wheat', 'Sugarcane', 'Miscellaneous']
    
    combinations = list(itertools.product(supply_params, demand_params))
    
    n_plots = len(combinations)
    n_cols = 5
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 5*n_rows))
    fig.suptitle('Supply vs Demand Parameters (Color: Annual Water Scarcity)', fontsize=16)
    
    for i, (supply_param, demand_param) in enumerate(combinations):
        row = i // n_cols
        col = i % n_cols
        ax = axs[row, col] if n_rows > 1 else axs[col]
        
        supply_index = supply_params.index(supply_param) + len(demand_params)
        demand_index = demand_params.index(demand_param)
        
        scatter = ax.scatter(all_lhs_samples[:, supply_index], 
                             all_lhs_samples[:, demand_index],
                             c=annual_scarcity, 
                             cmap='coolwarm')
        
        ax.set_xlabel(supply_param)
        ax.set_ylabel(demand_param)
        ax.set_title(f'{supply_param} vs {demand_param}')
        
        fig.colorbar(scatter, ax=ax, label='Annual Water Scarcity')
    
    for i in range(n_plots, n_rows * n_cols):
        row = i // n_cols
        col = i % n_cols
        fig.delaxes(axs[row, col] if n_rows > 1 else axs[col])
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(results_folder, 'supply_demand_combinations.png'))
    plt.close()

def plot_crop_vs_parameter(crop_demands, parameter_values, parameter_name, metrics, output_folder):
    metric_names = ['Annual Scarcity', 'Variability', 'Surplus Capacity', 'Deficit Severity', 'Storage Adequacy']
    crops = ['Cotton', 'Rice', 'Wheat', 'Sugarcane', 'Miscellaneous']
    
    for crop_index, crop in enumerate(crops):
        fig, axs = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle(f'{crop} Demand vs {parameter_name}', fontsize=16)
        axs = axs.flatten()
        
        for i, metric_name in enumerate(metric_names):
            ax = axs[i]
            scatter = ax.scatter(crop_demands[:, crop_index], parameter_values, c=metrics[metric_name], cmap='coolwarm')
            fig.colorbar(scatter, ax=ax, label=metric_name)
            ax.set_xlabel(f'{crop} Demand')
            ax.set_ylabel(parameter_name)
            ax.set_title(f'Color: {metric_name}')
        
        # Remove the unused subplot
        fig.delaxes(axs[5])
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f'{crop.lower()}_vs_{parameter_name.lower()}.png'))
        plt.close()

# Main execution
supply_flows_folder = './debugged'
var_metrics_folder = os.path.join(supply_flows_folder, '8. VAR_metrics')
supply_demand_folder = os.path.join(supply_flows_folder, '12. supply_demand_plots')
crop_parameter_folder = os.path.join(supply_flows_folder, '13. crop_parameter_plots')

for folder in [var_metrics_folder, supply_demand_folder, crop_parameter_folder]:
    assure_path_exists(folder)

var_metrics_file = os.path.join(var_metrics_folder, 'VAR_metrics.csv')
metrics_df = pd.read_csv(var_metrics_file)

# Extract metrics from metrics_df
metrics = {
    'Annual Scarcity': metrics_df['Annual_Scarcity'].values,
    'Variability': metrics_df['Variability_km3'].values,
    'Surplus Capacity': metrics_df['Surplus_Capacity_km3'].values,
    'Deficit Severity': metrics_df['Deficit_Severity_km3'].values,
    'Storage Adequacy': metrics_df['Storage_Adequacy'].values
}

# Load LHS samples
all_lhs_samples = np.loadtxt("LHsamples_all_params.txt")

# Create supply vs demand plots
plot_supply_demand_combinations(all_lhs_samples, metrics['Annual Scarcity'], supply_demand_folder)

# Create crop vs parameter plots
crop_demands = all_lhs_samples[:, :5]  # First 5 columns are crop demands
for i, param_name in enumerate(['Amplitude', 'Lambda']):
    param_values = all_lhs_samples[:, 5+i]
    plot_crop_vs_parameter(crop_demands, param_values, param_name, metrics, crop_parameter_folder)

print("Step 7 completed: Created Supply vs Demand plots")
print("Step 8 completed: Created Crop vs Parameter plots")

Step 7 completed: Created Supply vs Demand plots
Step 8 completed: Created Crop vs Parameter plots


In [ ]:
import numpy as np

def calculate_var_metrics_updated_storage(supply_data, demand_flows, current_storage, potential_storage):
    n_scenarios = len(supply_data)
    
    annual_scarcity = np.zeros(n_scenarios)
    surplus_capacity = np.zeros(n_scenarios)
    deficit_severity = np.zeros(n_scenarios)
    variability = np.zeros(n_scenarios)
    best_months = np.zeros(n_scenarios, dtype=int)
    worst_months = np.zeros(n_scenarios, dtype=int)
    current_storage_adequacy = np.zeros(n_scenarios)
    potential_storage_adequacy = np.zeros(n_scenarios)
    
    for i in range(n_scenarios):
        monthly_supply = np.mean(supply_data[i], axis=0)
        monthly_demand = np.sum([demand_flows[crop][i] for crop in demand_flows if crop != 'Total' and crop != 'All Scenarios Total'], axis=0)
        
        # Calculate annual scarcity
        annual_supply = np.sum(monthly_supply)
        annual_demand = np.sum(monthly_demand)
        annual_scarcity[i] = annual_demand / annual_supply
        
        extended_supply = np.concatenate([monthly_supply[-2:], monthly_supply, monthly_supply[:2]])
        extended_demand = np.concatenate([monthly_demand[-2:], monthly_demand, monthly_demand[:2]])
        
        # Calculate 3-month moving averages
        rolling_supply = np.convolve(extended_supply, np.ones(3), 'valid') / 3
        rolling_demand = np.convolve(extended_demand, np.ones(3), 'valid') / 3
        
        # Calculate rolling balance (average supply minus average demand)
        rolling_balance = rolling_supply - rolling_demand
        
        # Calculate variability
        best_balance = np.max(rolling_balance)
        worst_balance = np.min(rolling_balance)
        variability[i] = best_balance - worst_balance
        
        # Find best and worst months
        best_months[i] = np.argmax(rolling_balance) % 12
        worst_months[i] = np.argmin(rolling_balance) % 12
        
        # Calculate surplus capacity
        surplus_capacity[i] = best_balance
        
        # Calculate deficit severity
        deficit_severity[i] = -worst_balance
        
        # Calculate storage adequacy
        current_storage_adequacy[i] = 2 if variability[i] <= current_storage else (1 if variability[i] <= potential_storage else 0)
        potential_storage_adequacy[i] = 1 if variability[i] <= potential_storage else 0
    
    return annual_scarcity, variability, surplus_capacity, deficit_severity, best_months, worst_months, current_storage_adequacy, potential_storage_adequacy



current_storage = 14.3
potential_storage = current_storage + 18.0  # 32.3 BCM in total

annual_scarcity, variability, surplus_capacity, deficit_severity, best_months, worst_months, current_storage_adequacy, potential_storage_adequacy = calculate_var_metrics_updated_storage(supply_data, demand_flows, current_storage, potential_storage)

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import itertools

def assure_path_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Created directory: {path}")

def plot_supply_demand_combinations(all_lhs_samples, annual_scarcity, results_folder):
    assure_path_exists(results_folder)

    supply_params = ['Amplitude', 'lambda']
    demand_params = ['Cotton', 'Rice', 'Wheat', 'Sugarcane', 'Miscellaneous']
    
    combinations = list(itertools.product(supply_params, demand_params))
    
    n_plots = len(combinations)
    n_cols = 5
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 5*n_rows))
    fig.suptitle('Supply vs Demand Parameters (Color: Annual Water Scarcity)', fontsize=16)
    
    for i, (supply_param, demand_param) in enumerate(combinations):
        row = i // n_cols
        col = i % n_cols
        ax = axs[row, col] if n_rows > 1 else axs[col]
        
        supply_index = supply_params.index(supply_param) + len(demand_params)
        demand_index = demand_params.index(demand_param)
        
        scatter = ax.scatter(all_lhs_samples[:, supply_index], 
                             all_lhs_samples[:, demand_index],
                             c=annual_scarcity, 
                             cmap='coolwarm')
        
        ax.set_xlabel(supply_param)
        ax.set_ylabel(demand_param)
        ax.set_title(f'{supply_param} vs {demand_param}')
        
        fig.colorbar(scatter, ax=ax, label='Annual Water Scarcity')
    
    for i in range(n_plots, n_rows * n_cols):
        row = i // n_cols
        col = i % n_cols
        fig.delaxes(axs[row, col] if n_rows > 1 else axs[col])
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(results_folder, 'supply_demand_combinations.png'))
    plt.close()

def plot_crop_vs_parameter(crop_demands, parameter_values, parameter_name, metrics, output_folder):
    metric_names = ['Annual Scarcity', 'Variability', 'Surplus Capacity', 'Deficit Severity', 'Storage Adequacy']
    crops = ['Cotton', 'Rice', 'Wheat', 'Sugarcane', 'Miscellaneous']
    
    for crop_index, crop in enumerate(crops):
        fig, axs = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle(f'{crop} Demand vs {parameter_name}', fontsize=16)
        axs = axs.flatten()
        
        for i, metric_name in enumerate(metric_names):
            ax = axs[i]
            scatter = ax.scatter(crop_demands[:, crop_index], parameter_values, c=metrics[metric_name], cmap='coolwarm')
            fig.colorbar(scatter, ax=ax, label=metric_name)
            ax.set_xlabel(f'{crop} Demand')
            ax.set_ylabel(parameter_name)
            ax.set_title(f'Color: {metric_name}')
        
        # Remove the unused subplot
        fig.delaxes(axs[5])
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f'{crop.lower()}_vs_{parameter_name.lower()}.png'))
        plt.close()

# Main execution
supply_flows_folder = './debugged'
var_metrics_folder = os.path.join(supply_flows_folder, '8. VAR_metrics')
supply_demand_folder = os.path.join(supply_flows_folder, '12. supply_demand_plots')
crop_parameter_folder = os.path.join(supply_flows_folder, '13. crop_parameter_plots')

for folder in [var_metrics_folder, supply_demand_folder, crop_parameter_folder]:
    assure_path_exists(folder)

# Load VAR metrics
var_metrics_file = os.path.join(var_metrics_folder, 'VAR_metrics.csv')
metrics_df = pd.read_csv(var_metrics_file)

# Extract metrics from metrics_df
metrics = {
    'Annual Scarcity': metrics_df['Annual_Scarcity'].values,
    'Variability': metrics_df['Variability_km3'].values,
    'Surplus Capacity': metrics_df['Surplus_Capacity_km3'].values,
    'Deficit Severity': metrics_df['Deficit_Severity_km3'].values,
    'Storage Adequacy': metrics_df['Storage_Adequacy'].values
}

# Load LHS samples
all_lhs_samples = np.loadtxt("LHsamples_all_params.txt")

# Create supply vs demand plots
plot_supply_demand_combinations(all_lhs_samples, metrics['Annual Scarcity'], supply_demand_folder)

# Create crop vs parameter plots
crop_demands = all_lhs_samples[:, :5]  # First 5 columns are crop demands
for i, param_name in enumerate(['Amplitude', 'Lambda']):
    param_values = all_lhs_samples[:, 5+i]
    plot_crop_vs_parameter(crop_demands, param_values, param_name, metrics, crop_parameter_folder)

print("Step 7 completed: Created Supply vs Demand plots")
print("Step 8 completed: Created Crop vs Parameter plots")

Step 7 completed: Created Supply vs Demand plots
Step 8 completed: Created Crop vs Parameter plots
